# Тема курсовой: обучение модели классификации для прогнозирования не выполнения долговых обязательств по текущему кредину.

## 1. Подготовка ноутбука

- Подключаем библиотеки

In [ ]:
import pandas as pd
import numpy as np

from scipy.stats import shapiro, ttest_rel, chi2_contingency

from sklearn.metrics import r2_score, mean_squared_error as mse
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config Inlinebackend.figure_format = 'svg'

import warnings
warnings.filterwarnings('ignore')

##### Функции для подготовки данных

In [ ]:
def sampling(df: pd.DataFrame, target: str, columns: list, final=False) -> tuple:
    # формируем выборки
    df = df[columns]
    
    Xy_df = df[df[target].notna()]
    
    X = Xy_df.drop(target, axis=1)
    y = Xy_df[target]

    # сплитим выборку
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    if final:
        final = df[df[target].isna()]
        final = final.drop(target, axis=1)
        return X_train, X_test, y_train, y_test, final
    elif not final:
        return X_train, X_test, y_train, y_test

In [ ]:
def pred_feature(df, target, columns, num_leaves, max_depth, learning_rate, n_estimators, random_state=42):
        
    X_train, X_test, y_train, y_test, final = sampling(df=df, target=target, columns=columns, final=True)
        
    # обучаем модель
    xgb_model = LGBMRegressor(num_leaves=num_leaves, max_depth=max_depth, learning_rate=learning_rate, 
                                  n_estimators=n_estimators, random_state=random_state)
    xgb_model.fit(X_train, y_train)

    # предсказываем зачения
    y_train_pred = xgb_model.predict(X_train)
    y_test_pred = xgb_model.predict(X_test)

    # смотрим результат
    print(target)
    evaluation_result(y_train=y_train, y_train_pred=y_train_pred, y_test=y_test, y_test_pred=y_test_pred)
        
    df_pred = pd.DataFrame()
    df_pred[target] = xgb_model.predict(final)
    df_pred = df_pred.set_index(final.index)
        
    return df_pred

- Формируем пути до данных

In [ ]:
PATH_TO_TRAIN = '../input/course-project/course_project_train.csv'
PATH_TO_TEST = '../input/course-project/course_project_test.csv'

# PATH_TO_TRAIN = 'course_project_train.csv'
# PATH_TO_TEST = 'course_project_test.csv'

## 2. Анализ датасетов

##### Функции для анализа

In [ ]:
def missing_data(df: pd.DataFrame) -> pd.Series:
    """get dataframe and calculate emissions in its data in percentage"""
    
    featur_counts = df.count()
    object_counts = df.shape[0]
    
    calc_procent = (1 - featur_counts/object_counts) * 100
    
    feature_procent = round(calc_procent, 2)
            
    result = {i: f'{v}%' for i, v in feature_procent.items() if v != 0}
    
    return pd.Series(result)

In [ ]:
def correlation(
    df: pd.DataFrame, 
    target: str
) -> (print, plt.plot):
    """calculates correlation whith target and plots graph"""
    
    corr_table = df.corr()
    
    result = corr_table.drop([target], axis=0) if target != None else corr_table
    sort_values = result.sort_values(target, ascending=False)*100 if target != None else result
    
    coef = round(sort_values[target], 2)
    
    print(coef)
    
    plt.figure(figsize=(12,4))

    plt.barh(coef.index, coef.values)

    plt.title(f'Correlation with {target}', fontsize=16)
    plt.xlabel('percent', fontsize=14)
    
    plt.grid()

In [ ]:
def emission(
    series: pd.Series, 
    val: int = None
) -> print:
    """
    get series and shop emissions in it, 
    if val then can calculate emissions and is percent
    """
    print(
        f'min = {series.min()}\n'
        f'max = {series.max()}\n'
        f'mean = {series.mean()}'
    )
    if val != None:
        em = series > val

        print(
            f'Число выбрасов = {em.sum()}\n'
            f'Процент выбрасов = {round(em.sum() / series.shape[0] * 100, 2)}%'
        )
        
    plt.scatter(series.index, series.values)
    plt.plot()

In [ ]:
def chi2_test(df, feature, values, target='Credit Default'):
    
    alpha=0.5
    
    df = df[df[feature].notna()][:1000]

    df1 = df[df[feature] == values[0]]
    test1 = df1[target].value_counts()

    df2 = df[df[feature] == values[1]]
    test2 = df2[target].value_counts()
    
    p = chi2_contingency([test1, test2])[1]
    print(f'p = {p}')
    
    if p <= alpha:
        print(f'{values[0]} != {values[1]}, при alpha = {alpha}') 
    elif p > alpha:
        print(f'{values[0]} == {values[1]}, при alpha = {alpha}')

In [ ]:
def shapiro_test(feature: pd.Series) -> print:
    
    alpha = 0.5
    result = shapiro(feature[:1000])
    
    p = result[1]
    print(f'p = {p}')
    
    if p <= alpha:
        print(f'не имеет нормального распредение при alpha = {alpha}')
    elif p > alpha:
        print(f'имеет нормального распредение при alpha = {alpha}')

In [ ]:
def evaluation_result(y_train, y_train_pred, y_test, y_test_pred):
    
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)

    fig, ax = plt.subplots(nrows=1, ncols=2)
    ax1, ax2 = ax

    ax1.scatter(y_train, y_train_pred)
    ax1.set_title(f'r2_train = {round(r2_train, 3)}')
    ax1.set_xlabel('true')
    ax1.set_ylabel('pred')
    
    ax2.scatter(y_test, y_test_pred)
    ax2.set_title(f'r2_test = {round(r2_test, 3)}')
    ax2.set_xlabel('true')
    ax2.set_ylabel('pred')

    fig.set_size_inches(12, 4)
    plt.subplots_adjust(wspace=0.4, hspace=0.1)
    plt.show()

##### Описание датасета

* **Home Ownership** - домовладение
* **Annual Income** - годовой доход
* **Years in current job** - количество лет на текущем месте работы
* **Tax Liens** - налоговые обременения
* **Number of Open Accounts** - количество открытых счетов
* **Years of Credit History** - количество лет кредитной истории
* **Maximum Open Credit** - наибольший открытый кредит
* **Number of Credit Problems** - количество проблем с кредитом
* **Months since last delinquent** - количество месяцев с последней просрочки платежа
* **Bankruptcies** - банкротства
* **Purpose** - цель кредита
* **Term** - срок кредита
* **Current Loan Amount** - текущая сумма кредита
* **Current Credit Balance** - текущий кредитный баланс
* **Monthly Debt** - ежемесячный долг
* **Credit Score** - кредитный рейтинг
* **Credit Default** - факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)

##### Xy_df

In [ ]:
Xy_df = pd.read_csv(PATH_TO_TRAIN)
Xy_df.head(2)

- Состояние данных

In [ ]:
Xy_df.info()

Видно что в некоторых фичах есть __пропуски__<br>
- Просмотрим их целостность в __процентнах__:

In [ ]:
missing_data(df=Xy_df)

Видно что в некоторых фичах отсутствует __очень большое__ количество данных, от 5% до 54,5%
- Просмотрим сводку

In [ ]:
Xy_df.describe()

- Посмотрим __корреляцию__ с таргером

In [ ]:
correlation(df=Xy_df, target='Credit Default')

- Наибольшая __прямая зависимость__ с таргером наблюдается у _Credit Score_ = 44%<br>

(Скорее всего это __выбросы__, так как высокий __рейтинг__ не может совпадать с __не выполнением__ кредитвых обязательств.)

- Наибольшая __обратная зависимость__ наблюдается у _Current Load Amount_ = -23%<br> 

(Т.е. __высокий доход__ совпадает с __надежностью клиента__ по выполнению кредитных обязательств.)

##### X_Final

In [ ]:
X_final = pd.read_csv(PATH_TO_TEST)
X_final.head(2)

- Состояние данных

In [ ]:
X_final.info()

Видно что в некоторых фичах есть __пропуски__<br>
- Просмотрим их __процентное__ соотношение:

In [ ]:
missing_data(df=X_final)

Так же как и в train видно, что в некоторых фичах отсутствует __очень большое__ количество данных, от 3,5% до 54%

## 3. Обработка данных

##### Класс для предобработки данных

In [ ]:
class Preprocessing:
    
    def __init__(self, df, target, features):
        
        self.df = df
        self.target = target
        self.features = features
        
    def concat_values(self):
        
        df = self.df
        
        df.loc[df['Home Ownership'] == 'Have Mortgage', 'Home Ownership'] = 'Home Mortgage'
        df.loc[df['Home Ownership'] == 'Rent', 'Home Ownership'] = 'Own Home'
        
    def mark(self):
        
        df = self.df
        
        df.loc[df['Current Loan Amount'] < 2e7, 'emissionsCLA'] = 0
        df.loc[df['Current Loan Amount'] > 2e7, 'emissionsCLA'] = 1
        
        df.loc[df['Credit Score'] <= 999, 'errorsCS'] = 0
        df.loc[df['Credit Score'] > 999, 'errorsCS'] = 1
        df.loc[df['Credit Score'].isna(), 'errorsCS'] = 2
        
        df.loc[df['Annual Income'].notna(), 'omissionsAI'] = 0
        df.loc[df['Annual Income'].isna(), 'omissionsAI'] = 1
        
    def retype_in_float(self):
    
        df = self.df
        
        df['Home Ownership - float'] = df['Home Ownership'].map({'Home Mortgage': 0, 'Own Home': 1}).astype(float)
        self.features.append('Home Ownership - float')
        
        df['Term - float'] = df['Term'].map({'Short Term': 0, 'Long Term':1}).astype(float)
        self.features.append('Term - float')
        
    def missingMSLD(self):
        
        df = self.df
        
        df.loc[df['Months since last delinquent'].isna(), 'Months since last delinquent'] =\
            (df['Months since last delinquent'].isna()).sum()
        
    def missingBankruptcies(self):
        
        df = self.df
        
        df.loc[df['Bankruptcies'].isna(), 'Bankruptcies'] = df['Bankruptcies'].quantile(q=0.5)
        
    def missingAnIn(self):
        
        df = self.df

        target='Annual Income'
        
        pred = pred_feature(df=df, target=target, columns=self.features, 
                                 num_leaves=2, max_depth=1, learning_rate=0.1, n_estimators=3300)
        df.loc[df[target].isna(), target] = pred
        
        self.features.append(target)
        
    def missingCrSc(self):
        
        df = self.df
        
        target = 'Credit Score'
        
        pred = pred_feature(df=df, target=target, columns=self.features, 
                                 num_leaves=2, max_depth=1, learning_rate=0.1, n_estimators=1500)
        df.loc[df[target].isna(), target] = pred
        
        self.features.append(target)
        
    def emissionsCS(self):
        
        df = self.df
        
        df.loc[df['Credit Score']>999, 'Credit Score'] =\
            df.loc[df['Credit Score']>999, 'Credit Score'] / 10
        
    def emissionsMOC(self):
        
        df = self.df
        
        df.loc[df['Maximum Open Credit'] > 2e8, 'Maximum Open Credit'] = df['Maximum Open Credit'].mean()
        
    def emissionsCLA(self):
        
        df = self.df
        
        target = 'Current Loan Amount'
        
        df.loc[df[target] > 2e7, target] = np.nan
        
        pred = pred_feature(df=df, target=target, columns=self.features, 
                                 num_leaves=2, max_depth=1, learning_rate=0.1, n_estimators=470)
        df.loc[df[target].isna(), target] = pred
        
        self.features.append(target)
        
    def standard_data(self):
        
        df = self.df
        
        scaler = StandardScaler()
        df[self.features] = scaler.fit_transform(df[self.features])

        
    def pipline(self):
        
        self.drop_features()
        self.concat_values()
        self.mark()
        self.retype_in_float()
        
        self.missingMSLD()
        self.missingBankruptcies()
        self.missingYICJ()
        
        self.emissionsCS()
        self.emissionsMOC()
        
        self.missingAnIn()
        self.missingCrSc()
        
        self.emissionsCLA()
        self.standard_data()

In [ ]:
Xy_df.columns

### Отбираем признаки

In [ ]:
TARGET_NAME = 'Credit Default'
FEATURE_NAMES = ['Annual Income', 'Tax Liens', 'Number of Open Accounts', 'Years of Credit History', 'Maximum Open Credit', 
                 'Number of Credit Problems', 'Months since last delinquent', 'Bankruptcies', 'Current Loan Amount', 
                 'Current Credit Balance', 'Monthly Debt', 'Credit Score']

In [ ]:
xy_prep = Preprocessing(df=Xy_df, target=TARGET_NAME, features=FEATURE_NAMES) # инициализируем класс предобработки

### Проверка гипотез

##### Нормальность

- Просмотрим график __Annual Income__

In [ ]:
annInc = Xy_df[Xy_df['Annual Income'].notna()]['Annual Income']

sns.distplot(annInc)
plt.show()

$H_0 -$ __Annual Income__ имеет нормальное распределение <br>
$H_1 -$ __Annual Income__ не имеет нормального распределения

In [ ]:
shapiro_test(feature=annInc)

$H_0 = H_1,\ H_0$ опровергнута

- Просмотрим график __Monthly Debt__

In [ ]:
mountlyD = Xy_df['Monthly Debt']

sns.distplot(mountlyD)
plt.show()

$H_0 -$ __Monthly Debt__ имеет нормальное распределение<br>
$H_1 -$ __Monthly Debt__ не имеет нормального распределения

In [ ]:
shapiro_test(feature=mountlyD)

$H_0 = H_1,\ H_0$ опровергнута

##### Синхронность

- Посмотрим график классов таргета в __Home Ownership__

In [ ]:
sns.countplot(x='Home Ownership', hue='Credit Default', data=Xy_df)
plt.show()

$H_0 -$ Home Morgage и Have Mortgage - cинхронны<br>
$H_1 -$ Home Morgage и Have Mortgage - не cинхронны

In [ ]:
feature = 'Home Ownership'

values = 'Have Mortgage', 'Home Mortgage'
chi2_test(df=Xy_df, feature=feature, values=values)

values = 'Own Home', 'Rent'
chi2_test(df=Xy_df, feature=feature, values=values)

Составим функций для объединения значений и применим

In [ ]:
xy_prep.concat_values()

* ### Создание отметок и ретипизация (пропуски, выбросы, ретайпы)

In [ ]:
xy_prep.mark() # пропуски и выбросы
xy_prep.retype_in_float() # ретайпы

### Первичная обработка пропусков

In [ ]:
missing_data(df=Xy_df)

- **Months since last delinquent** - количество месяцев с последней просрочки платежа

Судя по большому количеству пропусков в этом признаке можно сделать вывод, что они говорят об __отсутствии просрочек__ по платежу, а значит посчитать количество месяцев не возможно.<br>
Тогда заполним пропуски __числом__, которое будет показывать нам - сколько __всего__ таких пропусков.

In [ ]:
xy_prep.missingMSLD()

- **Bankruptcies** - банкротство

Здесь ставнительно не много пропусков поэтому заполним их медиальным значением

In [ ]:
xy_prep.missingBankruptcies()

### Первичная обработка выбросов

- **Credit Score** - кредитный рейтинг

In [ ]:
emission(
    series=Xy_df['Credit Score'], 
    val=999,
)

Зная, что __максимальный__ кредитный рейтинг = 999, поделим выбросы на 10

In [ ]:
xy_prep.emissionsCS()

- **Maximum Open Credit** - максимальная сумма кредита

In [ ]:
emission(
    series=Xy_df['Maximum Open Credit'],
    val=2e8
)

Наблюдается 3 выброса, приведем их к __среднему__ значению

In [ ]:
xy_prep.emissionsMOC()

### Вторичная обработка пропусков

- **Annual Income** - годовой доход

Восстановим данные при помощи LightGBM

In [ ]:
xy_prep.missingAnIn()

- **Credit Score** - кредитный рейтинг

In [ ]:
xy_prep.missingCrSc()

### Вторичная обработка выбрасов

- **Current Loan Amount** - текущая сумма кредита

In [ ]:
emission(
    series=Xy_df['Current Loan Amount'],
    val=2e7,
)

Обработаем выбросы при помощи LightGBM

In [ ]:
xy_prep.emissionsCLA()

Просмотрим результаты

In [ ]:
correlation(df=Xy_df, target='Credit Default')

## Стандартизация данных

In [ ]:
xy_prep.standard_data()

## Разбиение на на train и test

In [ ]:
X = Xy_df[FEATURE_NAMES]
y = Xy_df[TARGET_NAME]